# `yacman` features and usage

This short tutorial show you the features of `yacman` package in action.

First, let's prepare some data to work with

In [1]:
import yaml
yaml_dict = {'cfg_version': 0.1, 'lvl1': {'lvl2': {'lvl3': {'entry': ['val1', 'val2']}}}}
yaml_str = """\
cfg_version: 0.1
lvl1:
  lvl2:
    lvl3:
      entry: ["val1","val2"]
"""
filepath = "test.yaml"

with open(filepath, 'w') as f:
    data = yaml.dump(yaml_dict, f)
    
import yacman

##  `YacAttMap` object creation

There are multiple ways to initialize an object of `YacAttMap` class:

1. **Read data from a YAML-formatted file**

In [2]:
yacmap = yacman.YacAttMap(filepath=filepath)
yacmap

cfg_version: 0.1
lvl1:
  lvl2:
    lvl3:
      entry: ['val1', 'val2']

2. **Read data from an `entries` mapping**

In [3]:
yacmap = yacman.YacAttMap(entries=yaml_dict)
yacmap

cfg_version: 0.1
lvl1:
  lvl2:
    lvl3:
      entry: ['val1', 'val2']

3. **Read data from a YAML-formatted string**

In [4]:
yacmap = yacman.YacAttMap(yamldata=yaml_str)
yacmap

cfg_version: 0.1
lvl1:
  lvl2:
    lvl3:
      entry: ['val1', 'val2']

## File locks; race-free writing
Instances of `YacAttMap` class support race-free writing and file locking, so that **it's safe to use them in multi-user contexts**

They can be created with or without write capabilities. Writable objects create a file lock, which prevents other processes managed by `yacman` from updating the source config file.

`writable` argument in the object constructor can be used to toggle writable mode. The source config file can be updated on disk (using `write` method) only if the `YacAttMap` instance is in writable mode

In [5]:
yacmap = yacman.YacAttMap(filepath=filepath, writable=False)


try:
    yacmap.write()
except OSError as e:
    print("Error caught: {}".format(e))

yacmap = yacman.YacAttMap(filepath=filepath, writable=True)
yacmap.write()


Error caught: You can't call write on an object that was created in read-only mode.


'/Users/mstolarczyk/Uczelnia/UVA/code/yacman/docs/test.yaml'

The write capabilities can be granted to an object:

In [6]:
yacmap = yacman.YacAttMap(filepath=filepath, writable=False)
yacmap.make_writable()
yacmap.write()

'/Users/mstolarczyk/Uczelnia/UVA/code/yacman/docs/test.yaml'

Or withheld:

In [7]:
yacmap.make_readonly()

True

If a file is currently locked by other `YacAttMap` object. The object will not be made writable/created with write capabilities until the lock is gone. If the lock persists, the action will fail (with a `RuntimeError`) after a selected `wait_time`, which is 10s by default:

In [ ]:
yacmap = yacman.YacAttMap(filepath=filepath, writable=True)

try:
    yacmap1 = yacman.YacAttMap(filepath=filepath, writable=True, wait_max=1)
except RuntimeError as e:
    print("\nError caught: {}".format(e))
    
try:
    yacmap1 = yacman.YacAttMap(filepath=filepath, writable=False, wait_max=1)
    yacmap1.make_writable()
except RuntimeError as e:
    print("\nError caught: {}".format(e))


Waiting for file lock: lock.test.yaml ..
Error caught: The maximum wait time has been reached and the lock file still exists.
Waiting for file lock: /Users/mstolarczyk/Uczelnia/UVA/code/yacman/docs/lock.test.yaml ..

Lastly, the `YacAttMap` class instances **can be used in a context manager**. This way the source config file will be locked, possibly updated (depending on what the user chooses to do), safely written to and unlocked with a single line of code:

In [ ]:
yacmap = yacman.YacAttMap(filepath=filepath)

with yacmap as y:
    y.test = "test"

yacmap1 = yacman.YacAttMap(filepath=filepath)
yacmap1